# scMetabolism

In [ ]:
library(scMetabolism)
library(Seurat)
library(ggplot2)
library(rsvd)
library(pheatmap)
library(dplyr)

In [ ]:
setwd("/disk213/xieqq/JINHUA138.sc/scMetabolism")

In [ ]:
pbmc = readRDS("/disk213/xieqq/JINHUA138.sc/RDS/Epithelial.rds")
pbmc <- subset(pbmc, subset = CellType %in% c('Enterocytes','Colonocytes'))

In [ ]:
countexp.Seurat <- sc.metabolism.Seurat(obj = pbmc,
                                        method = "AUCell",  #ssGSEA AUCell VISION GSVA
                                        imputation = F,
                                        ncores = 2,
                                        metabolism.type = "KEGG")

## 可视化

In [ ]:
library(ggplot2)
library(RColorBrewer)
library(reshape2)
library(ggforce)

In [ ]:
metabolism <- countexp.Seurat
df = data.frame(t(metabolism@assays[["METABOLISM"]][["score"]]))
write.csv(df, "scMetabolism.csv")

In [ ]:
df$orig.ident <- factor(df$orig.ident,levels=c("small_0","small_60","small_90","small_180","small_240","large_0","large_60","large_90","large_180","large_240"),
                        labels=c("01_small_0","02_small_60","03_small_90","04_small_180","05_small_240","06_large_0","07_large_60","08_large_90","09_large_180","10_large_240"))
avg_df <- aggregate(df[,1:84],list(df$orig.ident),mean)
rownames(avg_df) <- avg_df$Group.1
avg_df <- avg_df[,-1]
avg_df <- as.data.frame(t(avg_df))
write.csv(avg_df, "scMetabolism.matrix.SL.csv")

In [ ]:
data <- avg_df
kegg_inf <- read.csv("/disk213/xieqq/Software/KEGG/KEGG_map.csv", row.names=1, check.names=F)
metabolism_inf <- read.csv("scMetabolism.inf.csv", check.names=F)
metabolism_inf <- inner_join(metabolism_inf,kegg_inf[,c(1,4)],by=c("new_name"="KEGG"))

data <- subset(data,rownames(data) %in% metabolism_inf$old_name)
old_names <- metabolism_inf$old_name
new_names <- metabolism_inf$new_name
match_pos <- match(old_names, rownames(data))
rownames(data)[match_pos] <- new_names[match_pos]

annotation_row <- data.frame(Category=metabolism_inf$second_category)
rownames(annotation_row) <- metabolism_inf$new_name

annotation_col <- data.frame(Segment=sapply(strsplit(colnames(data), "-"), function(x) x[1]),
                             Time=sapply(strsplit(colnames(data), "-"), function(x) x[2]))
annotation_col$Time <- paste0(annotation_col$Time,"d")
rownames(annotation_col) <- colnames(data)

In [ ]:
breaks_values <- seq(-2, 2, length.out = 51) 
pheatmap(data, scale="row", 
         annotation_colors=ann_colors,
         annotation_row=annotation_row,
         show_rownames=T, show_colnames=T, cluster_rows=F, cluster_cols=F,
         color=colorRampPalette(c("navy",'white',"firebrick3"))(50),
         breaks=breaks_values, 
         filename="pheatmap.pdf", width=12, height=12)

In [ ]:
new_name="Carbohydrate metabolism"
input.pathway.new <- input.pathway[which(input.pathway$second_category==new_name),]

data <- matrix(data=NA,nrow=10,ncol=10,dimnames=list(sort(unique(input.pathway$orig.ident)),sort(unique(input.pathway$orig.ident))))
data <- as.data.frame(data)
data$group1 <- rownames(data)
data <- data %>% pivot_longer(cols=-group1, names_to="group2", values_to="p_value")

for (i in 1:100){
  group1=input.pathway.new$Value[which(input.pathway.new$orig.ident==data$group1[i])]
  group2=input.pathway.new$Value[which(input.pathway.new$orig.ident==data$group2[i])]
  wilcox_test_result <- wilcox.test(group1, group2)
  p_value <- wilcox_test_result$p.value
  data[i,3] <- p_value
}
data$adjusted_p_value <- p.adjust(data$p_value, method="BH") #aka "fdr"
data$`-logpadj` <- -log10(data$adjusted_p_value)
data[which(data$`-logpadj`==Inf),5] = max(data$`-logpadj`[which(data$`-logpadj`!=Inf)]+1)
data$type="na"
data$type[which(data$adjusted_p_value<0.00001)]="*****"
data$type[which(data$adjusted_p_value<0.0001&data$adjusted_p_value>=0.00001)]="****"
data$type[which(data$adjusted_p_value<0.001&data$adjusted_p_value>=0.0001)]="***"
data$type[which(data$adjusted_p_value<0.01&data$adjusted_p_value>=0.001)]="**"
data$type[which(data$adjusted_p_value<0.05&data$adjusted_p_value>=0.01)]="*"

out <- data[,c(1,2,5)] %>% pivot_wider(names_from="group2", values_from="-logpadj")
out = out[,-1]
rownames(out) <- colnames(out)
write.csv(out,paste0("-Log.Padj_ST_",new_name,".csv"))

Color <- brewer.pal(length(unique(data$type)), "Set2")
data$group1 <- rep(1:10, each = 10)
data$group2 <- rep(1:10, times = 10)

Pout <- ggplot(data, aes(x0=group1, y0=group2, fill=type, r=0.5)) +
  geom_circle(color="black", linewidth=0.25) +
  coord_fixed(ratio = 1)+
  scale_fill_manual(name="P-adjusted", 
                    values=c("*"="#66C2A5","**"="#FC8D62","***"="#8DA0CB","****"="#E78AC3","*****"="#A6D854","na"="#FFD92F"))+
  theme_bw()+ 
  theme(axis.text.x=element_text(color="black",size=16),
        axis.text.y=element_text(color="black",size=16),
        axis.title.y=element_text(color="black",size=16),
        legend.text=element_text(color="black",size=16),
        legend.title=element_text(color="black",size=16),
        panel.grid.major=element_blank(),
        panel.grid.minor=element_blank())
ggsave(filename=paste0("DotMatrixChart_ST_",new_name,".pdf"),plot=Pout,width=8,height=8)

In [ ]:
data <- read.csv("scMetabolism.matrix.SL.csv",check.names=F)
colnames(data)[1] <- "KEGG"
data <- data %>% pivot_longer(cols=-KEGG, names_to="group", values_to="value")
kegg_inf <- read.csv("/disk213/xieqq/Software/KEGG/KEGG_map.csv", row.names=1, check.names=F)
metabolism_inf <- read.csv("scMetabolism.inf.csv", check.names=F)
metabolism_inf <- inner_join(metabolism_inf,kegg_inf[,c(1,4)],by=c("new_name"="KEGG"))
data <- left_join(data,metabolism_inf,by=c("KEGG"="old_name"))
data <- subset(data, second_category %in% c("Carbohydrate metabolism","Energy metabolism","Lipid metabolism","Amino acid metabolism"))

analyse <- aggregate(data$value, by=list(group=data$group,second_category=data$second_category), mean)
analyse$second_category <- factor(analyse$second_category,levels=c("Carbohydrate metabolism","Energy metabolism","Lipid metabolism","Amino acid metabolism"))
write.csv(analyse, "scMetabolism.matrix.SL.line.csv")

Px <- ggplot(analyse, aes(x=group, y=x, group=second_category, color=second_category)) +
  geom_line(linewidth=1)+
  geom_point(size=2)+
  scale_colour_manual(values=c("#ccdf69","#54b45d","#2350a1","#a89f82"))+
  labs(x="", y="", title="", color="")+
  theme_bw()+
  theme(axis.text.x=element_text(angle=90,color="black",size=12),
        axis.text.y=element_text(color="black",size=12),
        axis.title.x=element_text(color="black",size=16),
        axis.title.y=element_text(color="black",size=16),
        legend.text=element_text(color="black",size=14),
        legend.title=element_text(color="black",size=14),
        panel.grid.major=element_blank(),
        panel.grid.minor=element_blank())
ggsave(filename="LinePlot.SL.pdf",plot=Px,width=10,height=7)
